# Set max_tokens to be close to the expected value

The guidance shared in this post was tested using GPT-4. It suggests that setting the max_tokens parameter close to the expected output tokens can help reduce the latency of requests.

https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/latency

The results were not conclusive, showing only minor differences between the two tests. 

It is important to consider- the intent of this technique is not to truncate responses that are longer than average. For example, if 90% of responses are less than 200 tokens, and 10% are much longer, setting the max_tokens to 200 tokens will give an improvement to response times. However, this may not give a useful response, as it will be cutoff part way. 

The expectation is that for responses that output a number of generation tokens below the max_token thershold

#### Load Helper Functions and Import Libraries

In [28]:
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# Load environment variables
load_dotenv()

import os
from openai import AzureOpenAI

# def aoai_call(system_message, prompt, model, max_tokens):
#     client = AzureOpenAI(
#         api_version=os.getenv("API_VERSION"),
#         azure_endpoint=os.getenv("AZURE_ENDPOINT"),
#         api_key=os.getenv("API_KEY")
#     )

#     start_time = time.time()

#     completion = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "system", "content": system_message},
#             {"role": "user", "content": prompt},
#         ],
#         max_tokens=max_tokens
#     )

#     end_time = time.time()
#     e2e_time = end_time - start_time

#     result = json.loads(completion.model_dump_json(indent=2))
#     prompt_tokens = result["usage"]["prompt_tokens"]
#     completion_tokens = result["usage"]["completion_tokens"]
#     completion_text = result["choices"][0]["message"]["content"]

#     return result, prompt_tokens, completion_tokens, completion_text, e2e_time
model=os.getenv("MODELGPT4-8k")

## Use case: RAG

In [3]:
context_documents="""

1. “Quantum Entanglement: Spooky Action at a Distance”
Abstract:
Quantum entanglement, a phenomenon that baffled even Einstein, lies at the heart of quantum mechanics. In this article, we delve into the mysterious world of entangled particles, exploring how they can be connected across vast distances instantaneously. From Bell’s theorem to quantum teleportation, we unravel the enigma of entanglement and its potential applications in quantum computing and secure communication.

Introduction:
Quantum entanglement defies classical intuition. Imagine two particles—say, electrons—created together and then separated by light-years. Remarkably, their properties remain intertwined, regardless of the distance between them. When one particle’s state changes, the other responds instantaneously, as if they share a hidden connection. But how does this “spooky action at a distance” work?

Bell’s Theorem:
Physicist John Bell proposed a test to determine whether entanglement was real or merely a statistical fluke. Experiments confirmed Bell’s predictions: the correlations between entangled particles violated classical limits. Quantum mechanics prevailed, and entanglement emerged as a fundamental property of the universe.

Quantum Teleportation:
Entanglement enables quantum teleportation—a process where information about one particle is transmitted to another, even if they are light-years apart. This isn’t “Star Trek” teleportation of matter; instead, it transfers quantum states. Researchers are harnessing this phenomenon for secure communication and quantum networks.

Applications:
Beyond teleportation, entanglement plays a pivotal role in quantum computing. Qubits, the building blocks of quantum computers, rely on entanglement for their power. Scientists are also exploring entanglement-based sensors, clocks, and cryptography.

2. “CRISPR-Cas9: Rewriting the Genetic Code”
Abstract:
CRISPR-Cas9, a revolutionary gene-editing tool, has transformed biology and medicine. In this article, we explore the origins of CRISPR, its mechanism, and its impact on genetic research. From curing genetic diseases to creating designer organisms, CRISPR opens new frontiers in biotechnology.

Introduction:
Clustered Regularly Interspaced Short Palindromic Repeats (CRISPR) were initially discovered in bacteria as part of their immune system. Scientists soon realized that they could repurpose this system for precise gene editing. Enter CRISPR-Cas9—the Swiss Army knife of genetic manipulation.

How It Works:
CRISPR-Cas9 acts like molecular scissors. It uses a guide RNA to target specific DNA sequences, and the Cas9 protein cuts the DNA at that location. Researchers can then insert, delete, or modify genes with unprecedented accuracy. The simplicity and efficiency of CRISPR have revolutionized genetic research.

Applications:
Treating Genetic Diseases: CRISPR holds promise for curing genetic disorders like sickle cell anemia and cystic fibrosis. Clinical trials are underway.
Agriculture: CRISPR can create crops resistant to pests, drought, and disease.
Designer Babies?: Ethical debates surround using CRISPR for human enhancement.
Conservation: CRISPR may help save endangered species by editing their genomes.

"""

### A: Setting max_tokens to 2000

**Time taken: 1.9 seconds**

In [34]:
# user_question=" What is Cripsr?"

# system_message="""
# You are a helpful AI assistant.
# """
# prompt=f"""
# context: {context_documents}
# Return the character "A".
# """

# e2e_times = []
# for _ in range(10):
#     result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model,2000)
#     print(f"Prompt Tokens: {prompt_tokens}")
#     print(f"Completion Tokens: {completion_tokens}")
#     print(f"Time taken: {e2e_time:.2f} seconds")
#     print(completion_text)
#     e2e_times.append(e2e_time)

# average_e2e_time = sum(e2e_times) / len(e2e_times)
# print(f"Average time taken: {average_e2e_time:.2f} seconds")

Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.94 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.66 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.82 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.58 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.99 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.90 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 2.58 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.78 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.70 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 2.09 seconds
A
Average time taken: 1.90 seconds


### B: Set max_tokens to 50 tokens

**Time taken: 1.7 seconds**

In [36]:
# user_question=" What is Cripsr?"

# system_message="""
# You are a helpful AI assistant.
# """
# prompt=f"""
# context: {context_documents}
# Return the character "A".
# """

# e2e_times = []
# for _ in range(10):
#     result,prompt_tokens,completion_tokens,completion_text,e2e_time=aoai_call(system_message,prompt,model,50)
#     print(f"Prompt Tokens: {prompt_tokens}")
#     print(f"Completion Tokens: {completion_tokens}")
#     print(f"Time taken: {e2e_time:.2f} seconds")
#     print(completion_text)
#     e2e_times.append(e2e_time)

# average_e2e_time = sum(e2e_times) / len(e2e_times)
# print(f"Average time taken: {average_e2e_time:.2f} seconds")

Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.85 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.79 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 2.43 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.74 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.48 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.70 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.57 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.96 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.52 seconds
A
Prompt Tokens: 675
Completion Tokens: 1
Total cost: $0.0406
Time taken: 1.52 seconds
A
Average time taken: 1.76 seconds


### A: TBC

**Time taken: 1.9 seconds**

In [23]:
# import os
# import time
# import json

# def aoai_call(system_message, prompt, model, max_tokens):
#     client = AzureOpenAI(
#         api_version=os.getenv("API_VERSION"),
#         azure_endpoint=os.getenv("AZURE_ENDPOINT"),
#         api_key=os.getenv("API_KEY")
#     )

#     start_time = time.time()

#     completion = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "system", "content": system_message},
#             {"role": "user", "content": prompt},
#         ],
#         stream=True,
#         max_tokens=max_tokens
#     )

#     end_time = time.time()
#     e2e_time = end_time - start_time

#     result = json.loads(completion.model_dump_json(indent=2))
#     prompt_tokens = result["usage"]["prompt_tokens"]
#     completion_tokens = result["usage"]["completion_tokens"]
#     completion_text = result["choices"][0]["message"]["content"]

#     return result, prompt_tokens, completion_tokens, completion_text, e2e_time

# # Example usage:
# system_message = "Welcome to the Azure OpenAI chat!"
# prompt = "Can you generate a creative poem about the moon?"
# max_tokens = 50
# result, _, _, generated_poem, _ = aoai_call(system_message, prompt, model, max_tokens)
# print(generated_poem)


AttributeError: 'Stream' object has no attribute 'model_dump_json'

In [37]:
# estimated_max_tokens=2000

# user_question=" What is crispr?"

# system_message="""
# You help write a summary of this document. Explain responses in detail.
# """
# prompt=f"""
# Context for answering the question:
# {context_documents}
# User question:
# {user_question}
# """

# client = AzureOpenAI(
#     api_version=os.getenv("API_VERSION"),
#     azure_endpoint=os.getenv("AZURE_ENDPOINT"),
#     api_key=os.getenv("API_KEY")
# )


# start_time = time.time()

# completion = client.chat.completions.create(
#     model=model,
#     messages=[
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": prompt},
#     ],
#     stream=True,
#     max_tokens=estimated_max_tokens
# )

# e2e_start_time = time.time()
# import time

# tbt_durations = []
# previous_time = time.time()


# for i, message in enumerate(completion):
#     if i==1:
#         time_to_first_token=time.time()-e2e_start_time

#     current_time = time.time()
#     tbt_durations.append(current_time - previous_time)
#     previous_time = current_time
#     # print(f"Message {i}: {message.choices[0].delta.content}")

# average_tbt_duration = sum(tbt_durations) / len(tbt_durations)
# # print(f"Average time between tokens: {average_tbt_duration} seconds")

# e2e_end_time = time.time()
# e2e_time = e2e_end_time - e2e_start_time




Message 0: None
Message 1: CR
Message 2: IS
Message 3: PR
Message 4: ,
Message 5:  also
Message 6:  known
Message 7:  as
Message 8:  CR
Message 9: IS
Message 10: PR
Message 11: -C
Message 12: as
Message 13: 9
Message 14: ,
Message 15:  is
Message 16:  a
Message 17:  revolutionary
Message 18:  gene
Message 19: -edit
Message 20: ing
Message 21:  tool
Message 22:  that
Message 23:  has
Message 24:  significantly
Message 25:  transformed
Message 26:  fields
Message 27:  like
Message 28:  biology
Message 29:  and
Message 30:  medicine
Message 31: .
Message 32:  The
Message 33:  term
Message 34:  "
Message 35: CR
Message 36: IS
Message 37: PR
Message 38: "
Message 39:  stands
Message 40:  for
Message 41:  Cluster
Message 42: ed
Message 43:  Regular
Message 44: ly
Message 45:  Int
Message 46: ers
Message 47: paced
Message 48:  Short
Message 49:  Pal
Message 50: ind
Message 51: rom
Message 52: ic
Message 53:  Re
Message 54: peats
Message 55: ,
Message 56:  which
Message 57:  were
Message 58:  

In [38]:
# print("End to end time: ", e2e_time)
# print("Time to first token: ", time_to_first_token)
# print("Average time between tokens: ", average_tbt_duration)

End to end time:  10.329296827316284
Time to first token:  1.4505589008331299
Average time between tokens:  0.046110647065298896


In [21]:
# # record the time before the request is sent
# start_time = time.time()

# # send a ChatCompletion request to count to 100
# response = client.chat.completions.create(
#     model=model,
#     messages=[
#         {'role': 'user', 'content': 'return the character "A" with no other characters.'}
#     ],
#     temperature=0,
#     stream=True  # again, we set stream=True
# )
# # create variables to collect the stream of chunks
# collected_chunks = []
# collected_messages = []
# # iterate through the stream of events
# for chunk in response:
#     chunk_time = time.time() - start_time  # calculate the time delay of the chunk
#     collected_chunks.append(chunk)  # save the event response
#     chunk_message = chunk.choices[0].delta.content  # extract the message
#     collected_messages.append(chunk_message)  # save the message
#     print(f"Message received {chunk_time:.2f} seconds after request: {chunk_message}")  # print the delay and text

# # print the time delay and text received
# print(f"Full response received {chunk_time:.2f} seconds after request")
# # clean None in collected_messages
# collected_messages = [m for m in collected_messages if m is not None]
# full_reply_content = ''.join([m for m in collected_messages])
# print(f"Full conversation received: {full_reply_content}")

Message received 1.34 seconds after request: None
Message received 1.37 seconds after request: A
Message received 1.37 seconds after request: None
Full response received 1.37 seconds after request
Full conversation received: A


In [56]:
import pandas as pd
client = AzureOpenAI(
    api_version=os.getenv("API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_key=os.getenv("API_KEY")
)

# Initialize DataFrame

def run_experiment(max_tokens,samples):
    df = pd.DataFrame(columns=['e2e_time', 'time_to_first_token', 'average_tbt_duration','achieved_completion_chunks'])
    

    user_question="What is crispr?"

    system_message="""
    You help write a summary of this document. Explain responses in detail.
    """
    prompt=f"""
    Context for answering the question:
    {context_documents}
    User question:
    {user_question}
    """
    for _ in range(samples):
        start_time = time.time()

        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt},
            ],
            stream=True,
            max_tokens=max_tokens
        )

        e2e_start_time = time.time()

        tbt_durations = []
        previous_time = time.time()

        for i, message in enumerate(completion):
            if i==1:
                time_to_first_token=time.time()-e2e_start_time

            current_time = time.time()
            tbt_durations.append(current_time - previous_time)
            previous_time = current_time

        average_tbt_duration = sum(tbt_durations) / len(tbt_durations)

        e2e_end_time = time.time()
        e2e_time = e2e_end_time - e2e_start_time
        # print(time_to_first_token)
        # print(tbt_durations)

        # Create a DataFrame for the current row
        current_df = pd.DataFrame({
            'e2e_time': [e2e_time],
            'time_to_first_token': [time_to_first_token],
            'average_tbt_duration': [average_tbt_duration],
            'achieved_completion_chunks': len(tbt_durations),
        })

        # Concatenate the current DataFrame with the main DataFrame
        df = pd.concat([df, current_df], ignore_index=True)

    # Print DataFrame
    return df




In [48]:
high_max_tokens_df=run_experiment(2000)

/var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_13895/2773086162.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, current_df], ignore_index=True)


     e2e_time  time_to_first_token  average_tbt_duration  \
0    6.809158             1.252731              0.041265   
1   10.628409             1.538930              0.058025   
2    6.501748             1.245632              0.034876   
3   10.789815             3.012136              0.065392   
4    5.754230             1.254783              0.035513   
5    6.065122             1.311185              0.037438   
6    6.913838             1.383482              0.041525   
7    6.175123             1.242275              0.038594   
8    5.657351             1.075053              0.030409   
9    8.330798             2.386161              0.054276   
10  20.318726             3.176152              0.083065   
11  10.647999             1.950534              0.070882   
12  13.326495             2.315608              0.076188   
13   7.042649             1.763406              0.052270   
14   6.672189             1.443635              0.040191   
15   4.724957             1.149270      

In [50]:
low_max_tokens_df=run_experiment(300)

/var/folders/04/byk1dtp95sn0gjlx1gsgdqkw0000gn/T/ipykernel_13895/2773086162.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, current_df], ignore_index=True)


     e2e_time  time_to_first_token  average_tbt_duration  \
0    8.998691             1.853194              0.062596   
1    8.529188             2.194671              0.048973   
2   12.616259             1.666113              0.067106   
3    7.767991             1.487084              0.042506   
4    7.624175             1.655109              0.048253   
5   12.936478             4.686015              0.071079   
6    7.431397             1.552279              0.046618   
7    7.317405             1.354394              0.041769   
8    8.854402             1.554690              0.047597   
9    5.599775             1.284790              0.036811   
10   9.313534             1.810169              0.050489   
11   7.723169             2.021707              0.052649   
12   8.578085             1.351509              0.045318   
13   7.665675             1.536036              0.043545   
14   9.132491             2.989842              0.054684   
15  12.035122             2.228785      

In [53]:
high_max_tokens_df.describe(percentiles = [.5, 0.9, .95, .99])

,e2e_time,time_to_first_token,average_tbt_duration
count,50.000000,50.000000,50.000000
mean,8.374658,1.718913,0.049460
std,3.231526,0.642517,0.015634
min,4.082051,0.889888,0.029366
50%,7.474675,1.502430,0.045805
90%,12.067254,2.886443,0.074296
95%,13.101746,3.095046,0.082644
99%,19.745233,3.317352,0.088437
max,20.318726,3.453015,0.089757


In [54]:
low_max_tokens_df.describe(percentiles = [.5, 0.9, .95, .99])

,e2e_time,time_to_first_token,average_tbt_duration
count,50.000000,50.000000,50.000000
mean,8.075564,1.692715,0.048814
std,1.864304,0.576373,0.009568
min,3.574954,0.877036,0.026676
50%,7.745580,1.609951,0.048547
90%,10.214790,2.152078,0.060033
95%,12.067068,2.431052,0.067509
99%,12.779571,3.854890,0.073375
max,12.936478,4.686015,0.075582
